# Lesson 4: Introduction to dendrites

## Lesson goals:

1. Discuss papers from last week focusing on spatial and temporal summation of synaptic input
2. Explore steady-state signal propagation from soma to dendrite
3. Explore action potential backpropagation from soma to dendrite
4. Inject synaptic current-shaped waveforms to explore transient signal propagation from dendrite to soma
5. Challenge - what happens to spike backpropagation in the distal dendrite if an EPSP precedes the spike by ~5 ms?
6. Papers for next week: the role of NMDA-type glutamate receptors in spatial and temporal input summation

## 1. Discuss papers from last week focusing on spatial and temporal summation of synaptic input

- Magee, J., Cook, E. (2000). Somatic EPSP amplitude is independent of synapse location in hippocampal pyramidal neurons Nature Neuroscience  3(9), 895-903. https://dx.doi.org/10.1038/78800
- Magee, J. Dendritic Ih normalizes temporal summation in hippocampal CA1 neurons. Nat Neurosci 2, 508–514 (1999). https://doi.org/10.1038/9158
- Bianchi, D., Marasco, A., Limongiello, A., Marchetti, C., Marie, H., Tirozzi, B., Migliore, M. (2012). On the mechanisms underlying the depolarization block in the spiking dynamics of CA1 pyramidal neurons Journal of Computational Neuroscience  33(2), 207-225. https://dx.doi.org/10.1007/s10827-012-0383-y

## 2. Explore steady-state signal propagation from soma to dendrite

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from collections import defaultdict
import math
from neuron import h
h.load_file('stdrun.hoc')

### Let's start with our optimized single compartment neuron from last week:

In [ ]:
sim_history = defaultdict(lambda: defaultdict(dict))

# obtained via coarse search with basinhopping followed by local polishing with simplex
best_x = soma_gnabar, soma_gkdrbar, soma_gkabar, soma_gl = [0.09389553, 0.02495658, 0.0331321,  0.00052055]  
bounds = [(0.1*xi, 10.*xi) for xi in best_x]

target = {}
target['f_I_slope'] = 200.  # Hz / nA
target['soma_R_inp'] = 150.  # MOhm


fit_line = lambda x, slope, intercept: x * slope + intercept


def simulate_f_I(stim_params):
    sim_summary_list = []
    step_current_stim.delay = stim_params['f_I']['delay']  # start time of current injection
    step_current_stim.dur = stim_params['f_I']['dur']  # duration in milliseconds
    for stim_amp in stim_params['f_I']['amp_list']:
        this_sim_summary = {}
        step_current_stim.amp = stim_amp
        this_sim_summary['stim_amp'] = stim_amp
        this_sim_summary['stim_delay'] = stim_params['f_I']['delay']
        this_sim_summary['stim_dur'] = stim_params['f_I']['dur']
        h.run()
        this_sim_summary['t'] = np.array(t)
        this_sim_summary['soma_voltage'] = np.array(soma_voltage)
        this_spike_times = np.array(spike_times)
        this_sim_summary['spike_times'] = this_spike_times
        
        # take only spike times occurring inside the current injection
        valid_spike_indexes = np.where((this_spike_times > f_I_stim_delay) & (this_spike_times <= f_I_stim_delay + f_I_stim_dur))[0]
        this_sim_summary['spike_count'] = len(valid_spike_indexes)
        this_sim_summary['spike_rate'] = len(valid_spike_indexes) / (f_I_stim_dur / 1000.)  # convert ms to s
        sim_summary_list.append(this_sim_summary)
    
    return sim_summary_list


def fit_f_I_curve(stim_amp, spike_rate):
    stim_amp_array = np.array(stim_amp)
    spike_rate_array = np.array(spike_rate)
    valid_stim_indexes = np.where(spike_rate_array > 0.)[0]
    if len(valid_stim_indexes) < 2:
        return 0., 0.
    slope, intercept, r_value, p_value, std_err = linregress(stim_amp_array[valid_stim_indexes], spike_rate_array[valid_stim_indexes])
    
    return slope, intercept


def get_f_I_slope_error(x, target, bounds, sim_history, stim_params):
    # enforce bounds:
    for xi, bi in zip(x, bounds):
        if not bi[0] <= xi <= bi[1]:
            return 1e9
    
    update_model(x)
    
    # simulate f_I and retrieve the data
    this_f_I_sim_summary_list = simulate_f_I(stim_params)
    
    # calculate the f_I slope
    this_spike_rate_list = [this_f_I_sim_summary['spike_rate'] for this_f_I_sim_summary in this_f_I_sim_summary_list]
    this_stim_amp_list = stim_params['f_I']['amp_list']
    this_f_I_slope, this_f_I_intercept = fit_f_I_curve(this_stim_amp_list, this_spike_rate_list)
    
    # calculate the squared error
    this_f_I_error = (this_f_I_slope - target['f_I_slope']) ** 2.
    
    sim_history[tuple(x)]['f_I']['data'] = this_f_I_sim_summary_list
    sim_history[tuple(x)]['f_I']['slope'] = this_f_I_slope
    sim_history[tuple(x)]['f_I']['intercept'] = this_f_I_intercept
    sim_history[tuple(x)]['f_I']['error'] = this_f_I_error
    
    return this_f_I_error


def calculate_R_inp(t, v, i_amp, baseline_window=(195., 200.),  measurement_window=(395., 400.)):
    baseline_indexes = np.where((baseline_window[0] <= t) & (t < baseline_window[1]))
    measurement_indexes = np.where((measurement_window[0] <= t) & (t < measurement_window[1]))
    baseline_v = np.mean(v[baseline_indexes])
    measurement_v = np.mean(v[measurement_indexes])
    delta_v = abs(baseline_v - measurement_v)
    R_inp = (delta_v / 1000.) / (abs(i_amp) / 1e9) / 1e6 # convert mV to V, nA to A, and Ohm to MegaOhm
    return R_inp


def simulate_soma_R_inp(stim_params):
    step_current_stim.delay = stim_params['R_inp']['delay']
    step_current_stim.dur = stim_params['R_inp']['dur']
    step_current_stim.amp = stim_params['R_inp']['amp']
    this_sim_summary = {}
    this_sim_summary['stim_delay'] = stim_params['R_inp']['delay']
    this_sim_summary['stim_dur'] = stim_params['R_inp']['dur']
    this_sim_summary['stim_amp'] = stim_params['R_inp']['amp']
    
    h.run()
    t_array = np.array(t)
    v_array = np.array(soma_voltage)
    soma_R_inp = calculate_R_inp(t_array, v_array, stim_params['R_inp']['amp'])
    
    this_sim_summary['t'] = t_array
    this_sim_summary['soma_voltage'] = v_array
    this_sim_summary['soma_R_inp'] = soma_R_inp
    
    return [this_sim_summary]  # for consistency with f_I functions, return a list of simulation summary dictionaries


def get_soma_R_inp_error(x, target, bounds, sim_history, stim_params):
    # enforce bounds
    for xi, bi in zip(x, bounds):
        if not bi[0] <= xi <= bi[1]:
            return 1e9
        
    update_model(x)
    
    this_R_inp_sim_summary_list = simulate_soma_R_inp(stim_params)
    this_soma_R_inp = this_R_inp_sim_summary_list[0]['soma_R_inp']
    
    # calculate the squared error
    this_R_inp_error = (this_soma_R_inp - target['soma_R_inp']) ** 2.
    
    sim_history[tuple(x)]['R_inp']['data'] = this_R_inp_sim_summary_list
    sim_history[tuple(x)]['R_inp']['soma_R_inp'] = this_soma_R_inp
    sim_history[tuple(x)]['R_inp']['error'] = this_R_inp_error
    
    return this_R_inp_error


def update_model(x):
    # parse the input parameters
    gnabar, gkdrbar, gkabar, gl = x
    
    soma(0.5).nax.gbar = gnabar
    soma(0.5).kdr.gkdrbar = gkdrbar
    soma(0.5).kap.gkabar = gkabar
    soma(0.5).pas.g = gl
    

def get_excitability_error(x, target, bounds, sim_history, stim_params):
    this_R_inp_error = get_soma_R_inp_error(x, target, bounds, sim_history, stim_params)
    this_f_I_slope_error = get_f_I_slope_error(x, target, bounds, sim_history, stim_params)
    
    return this_R_inp_error + this_f_I_slope_error

### ATTENTION: We forgot to set the axial resistivity to a physiological value last week!

In [ ]:
soma = h.Section()

# Set Ra to a physiological value
Ra0 = 150.  # Ohm-cm
soma.Ra = Ra0
soma.L = 20.
soma.diam = 20.
soma.insert('pas')  # leak
soma.insert('kdr')  # delayed-rectifier K+ channel
soma.insert('kap')  # A-type K+ channel
soma.insert('nax')  # Na+ channel

h.celsius = 35.  # These mechanisms are meant to work at (closer to) physiological temperature
h.tstop = 600.
v_init = -65.
h.v_init = v_init

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t, h.dt)  # record the time base
soma_voltage.record(soma(0.5)._ref_v, h.dt)  # record the voltage across the membrane in a segment

step_current_stim = h.IClamp(soma(0.5))
f_I_stim_dur = 200.
f_I_stim_delay = 200.
f_I_stim_amp_list = [0.05 * i for i in range(10)]

R_inp_stim_dur = 200.
R_inp_stim_delay = 200.
R_inp_stim_amp = -0.1

stim_params = {
    'f_I': {'delay': f_I_stim_delay, 'dur': f_I_stim_dur, 'amp_list': f_I_stim_amp_list},
    'R_inp': {'delay': R_inp_stim_delay, 'dur': R_inp_stim_dur, 'amp': R_inp_stim_amp}
              }

soma(0.5).pas.e = -65.
gl0 = soma(0.5).pas.g
gkdrbar0 = soma(0.5).kdr.gkdrbar
gkabar0 = soma(0.5).kap.gkabar
gnabar0 = soma(0.5).nax.gbar

spike_times = h.Vector()
spike_detector = h.NetCon(soma(0.5)._ref_v, None, sec=soma)
spike_detector.delay = 0.  # ms
spike_detector.threshold = -10.  # mV
spike_detector.record(spike_times)

update_model(best_x)

In [ ]:
sim_history = defaultdict(lambda: defaultdict(dict))

error = get_excitability_error(best_x, target, bounds, sim_history, stim_params)
print('x:', best_x, 'error:', error)

In [ ]:
fig, axis = plt.subplots()
for i, sim_summary in enumerate(sim_history[tuple(best_x)]['R_inp']['data']):
    axis.plot(sim_summary['t'], sim_summary['soma_voltage'])
    axis.set_ylabel('Voltage (mV)')
    axis.set_xlabel('Time (ms)')
fig.tight_layout()

print('soma_R_inp:', sim_history[tuple(best_x)]['R_inp']['soma_R_inp'], 'Error:', sim_history[tuple(best_x)]['R_inp']['error'])

fig, axes = plt.subplots(3, 4, figsize=(12., 8.))
for i, sim_summary in enumerate(sim_history[tuple(best_x)]['f_I']['data']):
    axes.flat[i].plot(sim_summary['t'], sim_summary['soma_voltage'])
    axes.flat[i].set_ylabel('Voltage (mV)')
    axes.flat[i].set_xlabel('Time (ms)')
fig.tight_layout()

this_slope = sim_history[tuple(best_x)]['f_I']['slope']
this_intercept = sim_history[tuple(best_x)]['f_I']['intercept']
stim_amp_array = np.array([sim_summary['stim_amp'] for sim_summary in sim_history[tuple(best_x)]['f_I']['data']])
spike_rate_array = np.array([sim_summary['spike_rate'] for sim_summary in sim_history[tuple(best_x)]['f_I']['data']])
valid_stim_indexes = np.where(spike_rate_array > 0.)[0]

plt.figure()
plt.scatter(stim_amp_array, spike_rate_array)
plt.plot(stim_amp_array[valid_stim_indexes], fit_line(stim_amp_array[valid_stim_indexes], this_slope, this_intercept))
plt.ylabel('Firing rate (Hz)')
plt.xlabel('Current amplitude (nA)')
plt.show()

print('f-I slope:', sim_history[tuple(best_x)]['f_I']['slope'], 'Error:', sim_history[tuple(best_x)]['f_I']['error'])

### Create a cylindrical dendrite section and connect it to the soma

In [ ]:
dend = h.Section()
dend.L = 300.
dend.diam = 2.
# Set Ra to the same value as the soma, it's a property of the cytoplasm
dend.Ra = Ra0

# parent section, parent location, child location
dend.connect(soma, 1., 0.)

### How many segments should we divide the section into?

### The "lambda" rule takes a desired signal frequency, and the dimensions of the compartment, and determines the "length constant":
 - https://en.wikipedia.org/wiki/Length_constant

In [ ]:
def lambda_f(sec, f=100.):
    """
    Calculates the AC length constant for the given section at the frequency f.
    :param sec: :class:'h.Section'
    :param f: float (Hz)
    :return: float (um)
    """
    diam = np.mean([seg.diam for seg in sec])
    Ra = sec.Ra
    cm = np.mean([seg.cm for seg in sec])
    return 1e5 * math.sqrt(diam / (4. * math.pi * f * Ra * cm))


dend_length_constant = lambda_f(dend)
print('dend length constant: %.3f' % dend_length_constant)

In [ ]:
def get_lambda_nseg(sec, lam=0.1, f=100.):
    """
    Used to determine the number of segments per hoc section to achieve the desired spatial and temporal resolution.
    Calculates the AC length constant for the given section at the frequency f.
    Assures that no segment will be larger than a fraction of the length_constant, specified by the proportion lam.
    :param sec : :class:'h.Section'
    :param lam : float: no segment will be larger than lam, a fraction of the section's length constant 
    :param f: float (Hz)
    :return : int
    """
    length_constant = lambda_f(sec, f)
    return int(((sec.L / (lam * length_constant)) + 0.9) / 2) * 2 + 1

dend_nseg = get_lambda_nseg(dend)
print('dend sec should have nseg=%i' % dend_nseg)

In [ ]:
dend.nseg = dend_nseg

### Let's set up a series of dendritic recording locations, one in every segment:

In [ ]:
dend_voltage_recs = []
dend_distances = []
for seg in dend:
    dend_voltage = h.Vector()
    loc = seg.x
    distance = loc * dend.L
    dend_voltage.record(dend(loc)._ref_v, h.dt)
    dend_voltage_recs.append(dend_voltage)
    dend_distances.append(distance)
    print('Recording from dendrite %.2f from soma' % distance)

### Let's inject current into the soma, and plot the recordings.

In [ ]:
step_current_stim.delay = R_inp_stim_delay
step_current_stim.dur = R_inp_stim_dur
step_current_stim.amp = R_inp_stim_amp
h.run()

fig, axis = plt.subplots()
axis.plot(t, soma_voltage, label='soma')
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    axis.plot(t, dend_voltage, label='dend (%.2f um)' % distance)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage (mV)')
axis.set_xlabel('Time (ms)')
fig.tight_layout()

### Why is there no attenutation from soma to dendrite?

In [ ]:
dend.insert('pas')
for seg in dend:
    seg.pas.g = soma_gl

### Let's try again with uniform leak throughout soma and dendrite:

In [ ]:
h.run()

fig, axis = plt.subplots()
axis.plot(t, soma_voltage, label='soma')
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    axis.plot(t, dend_voltage, label='dend (%.2f um)' % distance)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage (mV)')
axis.set_xlabel('Time (ms)')
fig.tight_layout()

### Let's plot the steady-state signal attenuation from soma to dendrite:

In [ ]:
def get_delta_v(t, v, baseline_window=(195., 200.),  measurement_window=(395., 400.)):
    baseline_indexes = np.where((baseline_window[0] <= t) & (t < baseline_window[1]))
    measurement_indexes = np.where((measurement_window[0] <= t) & (t < measurement_window[1]))
    baseline_v = np.mean(v[baseline_indexes])
    measurement_v = np.mean(v[measurement_indexes])
    delta_v = measurement_v - baseline_v
    return delta_v


t_array = np.array(t)
soma_v_array = np.array(soma_voltage)
soma_delta_v = get_delta_v(t_array, soma_v_array)

dend_attentuation = []
for i, dend_voltage in enumerate(dend_voltage_recs):
    this_delta_v = get_delta_v(t_array, np.array(dend_voltage))
    dend_attentuation.append(this_delta_v / soma_delta_v)
    
fig, axis = plt.subplots()
axis.plot(dend_distances, dend_attentuation)
axis.set_ylabel('Voltage attenuation (soma -> dend)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### What is the effect of changing dendritic leak conductance (input resistance)?

### Here's another strategy for saving and labeling simulations:

In [ ]:
sim_history_data = []
sim_history_labels = []


def run_sim(label):
    h.run()
    sim_summary = {}
    sim_summary['t'] = np.array(t)
    sim_summary['soma_voltage'] = np.array(soma_voltage)
    dend_voltage_list = []
    for dend_voltage_rec in dend_voltage_recs:
        dend_voltage_list.append(np.array(dend_voltage_rec))
    sim_summary['dend_voltage_list'] = dend_voltage_list
    sim_history_labels.append(label)
    sim_history_data.append(sim_summary)

In [ ]:
run_sim('default dend leak')

In [ ]:
for seg in dend:
    seg.pas.g = 0.5 * soma_gl
run_sim('0.5x dend leak')

In [ ]:
for seg in dend:
    seg.pas.g = 2. * soma_gl
run_sim('2x dend leak')

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    soma_delta_v = get_delta_v(data['t'], data['soma_voltage'])
    dend_attentuation = []
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        this_delta_v = get_delta_v(data['t'], dend_voltage)
        dend_attentuation.append(this_delta_v / soma_delta_v)
    axis.plot(dend_distances, dend_attentuation, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage attenuation (soma -> dend)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### We want closer to 50% attenuation at ~300 um from the soma:

In [ ]:
for seg in dend:
    seg.pas.g = 1.5 * soma_gl
run_sim('1.5x dend leak')

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    soma_delta_v = get_delta_v(data['t'], data['soma_voltage'])
    dend_attentuation = []
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        this_delta_v = get_delta_v(data['t'], dend_voltage)
        dend_attentuation.append(this_delta_v / soma_delta_v)
    axis.plot(dend_distances, dend_attentuation, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage attenuation (soma -> dend)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

## 3. Explore action potential backpropagation from soma to dendrite

### Let's inject a brief, large current to get one spike:

In [ ]:
step_current_stim.dur = 1.
step_current_stim.amp = 1.
h.run()

fig, axis = plt.subplots()
axis.plot(t, soma_voltage, label='soma')
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    axis.plot(t, dend_voltage, label='dend (%.2f um)' % distance)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage (mV)')
axis.set_xlabel('Time (ms)')
axis.set_xlim((195., 210.))
fig.tight_layout()

### Let's quantify the spike height:

In [ ]:
def get_event_peak_amp(t, v, baseline_window=(195., 200.),  measurement_window=(200., 205.), direction=1.):
    baseline_indexes = np.where((baseline_window[0] <= t) & (t < baseline_window[1]))
    measurement_indexes = np.where((measurement_window[0] <= t) & (t < measurement_window[1]))
    baseline_v = np.mean(v[baseline_indexes])
    if direction > 0.:
        measurement_v = np.max(v[measurement_indexes])
    else:
        measurement_v = np.min(v[measurement_indexes])
    delta_v = measurement_v - baseline_v
    return delta_v

In [ ]:
fig, axis = plt.subplots()

spike_height = []
these_distances = []
t_array = np.array(t)
spike_height.append(get_event_peak_amp(t_array, np.array(soma_voltage)))
these_distances.append(0.)
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    these_distances.append(distance)
    this_spike_height = get_event_peak_amp(t_array, np.array(dend_voltage))
    spike_height.append(this_spike_height)
axis.plot(these_distances, spike_height)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Spike amplitude (mV)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### What if dendrites are endowed with the same active channels that drive spiking in the soma?

In [ ]:
# save a record of a simulation with passive dendrites first
sim_history_data = []
sim_history_labels = []

run_sim('passive dend')

In [ ]:
dend.insert('nax')
dend.insert('kdr')
for seg in dend:
    seg.nax.gbar = 0.1 * soma_gnabar
    seg.kdr.gkdrbar = soma_gkdrbar

In [ ]:
run_sim('active dend (no ka)')

### First let's look at the traces:

In [ ]:
fig, axis = plt.subplots()
axis.plot(t, soma_voltage, label='soma')
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    axis.plot(t, dend_voltage, label='dend (%.2f um)' % distance)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage (mV)')
axis.set_xlabel('Time (ms)')
axis.set_xlim((195., 210.))
fig.tight_layout()

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    spike_height = []
    these_distances = []
    spike_height.append(get_event_peak_amp(data['t'], data['soma_voltage']))
    these_distances.append(0.)
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        distance = dend_distances[i]
        these_distances.append(distance)
        this_spike_height = get_event_peak_amp(data['t'], dend_voltage)
        spike_height.append(this_spike_height)
    axis.plot(these_distances, spike_height, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Spike amplitude (mV)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### This resembles the Hoffman et al., 1997 result in the presence of 4-AP to block A-type K+ channels

So what happens if we insert Ka?

In [ ]:
dend.insert('kap')
for seg in dend:
    seg.kap.gkabar = soma_gkabar

In [ ]:
run_sim('active dend (uniform ka)')

In [ ]:
fig, axis = plt.subplots()
axis.plot(t, soma_voltage, label='soma')
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    axis.plot(t, dend_voltage, label='dend (%.2f um)' % distance)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage (mV)')
axis.set_xlabel('Time (ms)')
axis.set_xlim((195., 210.))
fig.tight_layout()

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    spike_height = []
    these_distances = []
    spike_height.append(get_event_peak_amp(data['t'], data['soma_voltage']))
    these_distances.append(0.)
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        distance = dend_distances[i]
        these_distances.append(distance)
        this_spike_height = get_event_peak_amp(data['t'], dend_voltage)
        spike_height.append(this_spike_height)
    axis.plot(these_distances, spike_height, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Spike amplitude (mV)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### Should we increase Ka?

In [ ]:
for seg in dend:
    seg.kap.gkabar = 2. * soma_gkabar

In [ ]:
run_sim('active dend (2x ka in dend)')

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    spike_height = []
    these_distances = []
    spike_height.append(get_event_peak_amp(data['t'], data['soma_voltage']))
    these_distances.append(0.)
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        distance = dend_distances[i]
        these_distances.append(distance)
        this_spike_height = get_event_peak_amp(data['t'], dend_voltage)
        spike_height.append(this_spike_height)
    axis.plot(these_distances, spike_height, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Spike amplitude (mV)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### More?

In [ ]:
for seg in dend:
    seg.kap.gkabar = 10. * soma_gkabar

In [ ]:
run_sim('active dend (10x ka in dend)')

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    spike_height = []
    these_distances = []
    spike_height.append(get_event_peak_amp(data['t'], data['soma_voltage']))
    these_distances.append(0.)
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        distance = dend_distances[i]
        these_distances.append(distance)
        this_spike_height = get_event_peak_amp(data['t'], dend_voltage)
        spike_height.append(this_spike_height)
    axis.plot(these_distances, spike_height, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Spike amplitude (mV)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

### What if Ka is expressed in a linear gradient into dendrites?

In [ ]:
ka_gradient = lambda distance, slope: soma_gkabar + slope * distance

slope = 1.5 * soma_gkabar / 100.  # double the somatic conductance density in 100 um

for seg in dend:
    distance = seg.x * dend.L
    seg.kap.gkabar = ka_gradient(distance, slope)

In [ ]:
run_sim('active dend (linear ka gradient in dend)')

In [ ]:
fig, axis = plt.subplots()
for label, data in zip(sim_history_labels, sim_history_data):
    spike_height = []
    these_distances = []
    spike_height.append(get_event_peak_amp(data['t'], data['soma_voltage']))
    these_distances.append(0.)
    for i, dend_voltage in enumerate(data['dend_voltage_list']):
        distance = dend_distances[i]
        these_distances.append(distance)
        this_spike_height = get_event_peak_amp(data['t'], dend_voltage)
        spike_height.append(this_spike_height)
    axis.plot(these_distances, spike_height, label=label)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Spike amplitude (mV)')
axis.set_xlabel('Distance to soma (um)')
fig.tight_layout()

## 4. Inject synaptic current-shaped waveforms to explore transient signal propagation from dendrite to soma

### Let's turn off our somatic current injection and add one to the middle of the dendrite:

In [ ]:
step_current_stim.amp = 0.

dend_current_stim = h.IClamp(dend(0.5))

### But instead of injecting a square step current, let's construct a waveform to inject:

In [ ]:
rise_and_decay = np.exp(-(t_array - 200.) / 10.) - np.exp(-(t_array - 200.) / 0.5)
rise_and_decay[np.where(t_array < 200.)[0]] = 0.
rise_and_decay /= np.max(rise_and_decay)
plt.figure()
plt.plot(t_array, rise_and_decay)

In [ ]:
dend_current_stim.dur = 1e9  # the waveform will determine the duration
dend_current_stim.delay = 0.  # the baseline is now built in to the waveform

dend_stim_waveform_amp = 0.05 # nA
dend_stim_amp_vector = h.Vector(dend_stim_waveform_amp * rise_and_decay)
dend_stim_t_vector = h.Vector(t_array) 
dend_stim_amp_vector.play(dend_current_stim._ref_amp, dend_stim_t_vector)

In [ ]:
h.run()

fig, axis = plt.subplots()
axis.plot(t, soma_voltage, label='soma')
for i, dend_voltage in enumerate(dend_voltage_recs):
    distance = dend_distances[i]
    axis.plot(t, dend_voltage, label='dend (%.2f um)' % distance)
axis.legend(loc='best', frameon=False, framealpha=0.5)
axis.set_ylabel('Voltage (mV)')
axis.set_xlabel('Time (ms)')
axis.set_xlim((190., 300.))
fig.tight_layout()

### Let's quantify the amplitude and duration (full width at half max) at each recording location:

In [ ]:
def get_event_shape(t, v, baseline_window=(195., 200.),  measurement_window=(200., 300.), direction=1.):
    baseline_indexes = np.where((baseline_window[0] <= t) & (t < baseline_window[1]))
    measurement_indexes = np.where((measurement_window[0] <= t) & (t < measurement_window[1]))
    baseline_v = np.mean(v[baseline_indexes])
    subtracted_v = v - baseline_v
    if direction > 0.:
        amp = np.max(subtracted_v[measurement_indexes])
    else:
        amp = np.min(subtracted_v[measurement_indexes])
    rise_loc = np.where(subtracted_v[measurement_indexes] >= 0.5 * amp)[0][0]  # the first index that crosses 50% amplitude
    decay_loc = np.where(subtracted_v[measurement_indexes] >= 0.5 * amp)[0][-1]  # the last index that crosses 50% amplitude
    FWHM = t[measurement_indexes][decay_loc] - t[measurement_indexes][rise_loc]
    return amp, FWHM

In [ ]:
event_amp = []
these_distances = []
FWHM = []
t_array = np.array(t)
soma_amp, soma_FWHM = get_event_shape(t_array, np.array(soma_voltage))
event_amp.append(soma_amp)
FWHM.append(soma_FWHM)
these_distances.append(0.)

for i, dend_voltage_rec in enumerate(dend_voltage_recs):
    this_amp, this_FWHM = get_event_shape(t_array, np.array(dend_voltage_rec))
    event_amp.append(this_amp)
    FWHM.append(this_FWHM)
    these_distances.append(dend_distances[i])
    
fig, axes = plt.subplots(1, 2)
axes[0].plot(these_distances, event_amp)
axes[1].plot(these_distances, FWHM)
axes[0].set_title('EPSP amplitude')
axes[1].set_title('EPSP duration')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_ylabel('Amplitude (mV)')
axes[1].set_ylabel('Duration (ms)')
axes[0].set_xlabel('Distance to soma (um)')
axes[1].set_xlabel('Distance to soma (um)')
fig.tight_layout()

### Here's an example of using just one dendritic recording and stimulus to sequentially sample all locations:

In [ ]:
soma_amp_list = []
soma_FWHM_list = []
local_amp_list = []
local_FWHM_list = []

fig, axes = plt.subplots(2, 2, figsize=(10., 8.))
for i, seg in enumerate(dend):
    loc = seg.x
    this_dend_voltage_rec = h.Vector()
    this_dend_voltage_rec.record(dend(loc)._ref_v, h.dt)
    dend_current_stim = h.IClamp(dend(loc))
    dend_current_stim.dur = 1e9
    dend_current_stim.delay = 0.
    dend_stim_amp_vector.play(dend_current_stim._ref_amp, dend_stim_t_vector)
    h.run()
    t_array = np.array(t)
    soma_voltage_array = np.array(soma_voltage)
    this_soma_amp, this_soma_FWHM = get_event_shape(t_array, soma_voltage_array)
    soma_amp_list.append(this_soma_amp)
    soma_FWHM_list.append(this_soma_FWHM)
    dend_voltage_array = np.array(this_dend_voltage_rec)
    this_dend_amp, this_dend_FWHM = get_event_shape(t_array, dend_voltage_array)
    local_amp_list.append(this_dend_amp)
    local_FWHM_list.append(this_dend_FWHM)
    axes[0][0].plot(t_array, soma_voltage_array, label='Stim loc: %.2f um' % dend_distances[i])
    axes[0][1].plot(t_array, dend_voltage_array)
axes[0][0].set_ylabel('Voltage (mV)')
axes[0][1].set_ylabel('Voltage (mV)')
axes[0][0].set_xlabel('Time (ms)')
axes[0][1].set_xlabel('Time (ms)')
axes[0][0].set_xlim((190., 300.))
axes[0][1].set_xlim((190., 300.))
axes[0][0].set_title('Soma Vm')
axes[0][1].set_title('Local dend Vm')
axes[0][0].legend(loc='best', frameon=False, framealpha=0.5)
axes[1][0].plot(dend_distances, soma_amp_list, label='Soma')
axes[1][0].plot(dend_distances, local_amp_list, label='Dend')
axes[1][1].plot(dend_distances, soma_FWHM_list)
axes[1][1].plot(dend_distances, local_FWHM_list)
axes[1][0].set_title('EPSP amplitude')
axes[1][0].set_ylabel('Amplitude (mV)')
axes[1][0].set_xlabel('Distance to soma (um)')
axes[1][0].legend(loc='best', frameon=False, framealpha=0.5)
axes[1][1].set_title('EPSP duration')
axes[1][1].set_ylabel('Duration (ms)')
axes[1][1].set_xlabel('Distance to soma (um)')
fig.tight_layout()
fig.show()

## 5. Challenge - what happens to spike backpropagation in the distal dendrite if an EPSP precedes the spike by ~5 ms?

## 6. Papers for next week: the role of NMDA-type glutamate receptors in spatial and temporal input summation

- Losonczy, A., Magee, J. (2006). Integrative Properties of Radial Oblique Dendrites in Hippocampal CA1 Pyramidal Neurons Neuron  50(2), 291-307. https://dx.doi.org/10.1016/j.neuron.2006.03.016
- Popescu, G., Robert, A., Howe, J., Auerbach, A. (2004). Reaction mechanism determines NMDA receptor response to repetitive stimulation Nature  430(7001), 790-793. https://dx.doi.org/10.1038/nature02775